In [ ]:
import pickle
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from IPython import display
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.image import resize, resize_with_pad
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Embedding, Add, LSTM
from tensorflow.keras.models import Model, save_model, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tqdm.notebook import tqdm
import pandas as pd

## Preparing test images and captions

In [ ]:
df_test = df_test.reset_index()

In [ ]:
images_index = np.random.randint(len(df_test.id.unique()), size=5)

In [ ]:
test_images = [df_test.id.unique()[i] for i in images_index]
test_images

In [ ]:
true_captions = []
for imgs in test_images:
    image_captions = []
    for index, row in df_test.iterrows():
        if imgs in row['id']:
            image_captions.append(row['value'].split('startsequence ')[1])
    true_captions.append(image_captions)

In [ ]:
index_file ='/home/jupyter/robo_romeo//raw_data/word_index'
with open(index_file,'rb') as dictionary:
    word_index = pickle.load(dictionary)

In [ ]:
model_file = '/home/jupyter/robo_romeo/saved_models/model_run_20220611_1000'
predict_model = load_model(model_file)

In [ ]:
predictions = []
for imgs in test_images:
    test_img_path = '/home/jupyter/robo_romeo/raw_data/images/images/'+imgs+'.jpg'
    test_img = image.load_img(test_img_path, target_size=(256,256,3))
    x_test = image.img_to_array(test_img)
    x_test = np.expand_dims(x_test, axis=0)
    test = cnn_model.predict(x_test)

    inputs_seq = 'startsequence'
    img_encoded = test
    inputs_seq = [2]
    for i in range(36):
        inputs_seq_model = pad_sequences([inputs_seq],padding='post',maxlen=36)
        y_pre = predict_model.predict([img_encoded,inputs_seq_model])
        next_word = y_pre.argmax()
        if next_word == 13:
            break
        inputs_seq.append(next_word)

    sentence = []
    for number in inputs_seq:
        sentence.append(list(word_index.keys())[list(word_index.values()).index(number)])
    sentence = sentence [1:]
    prediction = ' '.join(word for word in sentence)
    predictions.append(prediction)

In [ ]:
fig = plt.figure(figsize=(16,12))
for i, images in enumerate(test_images):
    plt.subplot(5,5,i+1)
    img = plt.imread(f'/home/jupyter/robo_romeo/raw_data/images/images/{images}.jpg')
    plt.imshow(img)